In [1]:
from genericpath import exists
import pdb
from pickletools import uint8
from turtle import pd
import torchvision
import torch


import os
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
# from torchknickknacks import modelutils
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
from pytorch_grad_cam.utils.image import show_cam_on_image

import numpy as np
import cv2
import requests
from pytorch_grad_cam.ablation_layer import AblationLayerFasterRCNN
from pytorch_grad_cam.utils.model_targets import FasterRCNNBoxScoreTarget
from pytorch_grad_cam.utils.reshape_transforms import fasterrcnn_reshape_transform
from pytorch_grad_cam.ablation_layer import AblationLayerFasterRCNN
import random
import glob
from skimage.measure import label, regionprops, regionprops_table
from skimage import data, filters, measure, morphology
import pandas as pd
import wandb
from tqdm import tqdm
from skimage import data
from skimage.color import rgb2hed, hed2rgb
import sys
sys.path.insert(0, '../')
#from models.model_mrcnn import _default_mrcnn_config, build_default
from models_pytorch_lightning.model_mrcnn_config import _default_mrcnn_config, build_default
from features import build_features
from data.test_data import tiling_nosaving
from models_pytorch_lightning.generalized_mask_rcnn_pl import LitMaskRCNN
from utils.helper_functions import evaluate_metrics, get_outputs, compute_iou, evaluate_mask_rcnn
from features import build_features


In [6]:
class ExplainPredictions():
    # TODO fix the visualization flags
    def __init__(self, model, model_input_path, slide_dir,tile_dir, detection_threshold, wandb, save_result, ablation_cam, save_thresholds):
        self.model = model
        self.model_input_path = model_input_path
        self.slide_dir = slide_dir
        self.tile_dir = tile_dir
        self.detection_threshold = detection_threshold
        self.wandb = wandb
        self.save_result = save_result
        self.ablation_cam = ablation_cam
        self.save_thresholds = save_thresholds
        self.class_names = ['Cored', 'Diffuse', 'Coarse-Grained', 'CAA']
        #self.class_to_colors = {'Core': (255, 0, 0), 'Neuritic' : (0, 0, 255), 'Diffuse': (0,255,0), 'CAA':(225, 255, 0)}
        self.class_to_colors = {'Cored': (255, 0, 0), 'Diffuse' : (0, 0, 255), 'Coarse-Grained': (0,255,0), 'CAA':(225, 255, 0)}
        #self.result_save_dir= "/mnt/new-nas/work/data/npsad_data/vivek/reports/figures/"
        self.result_save_dir= os.path.join( "/home/mahirwar/Desktop/Monika/npsad_data/vivek/reports/Full-Minerva-Data-test-output_1", self.model_input_path.split("/")[-1])
        self.colors = np.random.uniform(0, 255, size=(len(self.class_names), 3))
        self.column_names = ["image_name", "region", "region_mask", "label", 
                            "confidence", "brown_pixels", "centroid", 
                            "eccentricity", "area", "equivalent_diameter","mask_present"]
        self.results_path = ""
        self.masks_path = "" 
        self.detections_path = "" 
        self.ablations_path = ""
        self.quantify_path = ""
      
    def get_brown_pixel_cnt(self, img, img_name):

        # Separate the stains from the IHC image
        ihc_hed = rgb2hed(img)

        # Create an RGB image for each of the stains
        null = np.zeros_like(ihc_hed[:, :, 0])
        ihc_h = hed2rgb(np.stack((ihc_hed[:, :, 0], null, null), axis=-1))
        ihc_e = hed2rgb(np.stack((null, ihc_hed[:, :, 1], null), axis=-1))
        ihc_d = hed2rgb(np.stack((null, null, ihc_hed[:, :, 2]), axis=-1))
        ihc_d = ihc_d.astype('float32')

        gray = cv2.cvtColor(ihc_d, cv2.COLOR_RGB2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)
        x = gray[gray<0.35]


        if len(x) != 0:
            if self.save_thresholds:
                # Display
                fig, axes = plt.subplots(2, 2, figsize=(7, 6), sharex=True, sharey=True)
                ax = axes.ravel()

                ax[0].imshow(img)
                ax[0].set_title("Original image")

                ax[1].imshow(ihc_h)
                ax[1].set_title("Hematoxylin")

                ax[2].imshow(gray)
                ax[2].set_title("gray")  # Note that there is no Eosin stain in this image

                ax[3].imshow(ihc_d)
                ax[3].set_title("DAB")

                for a in ax.ravel():
                    a.axis('off')

                fig.tight_layout()
            
                
                final_save_path = img_name + "_count_threhold.png"
                final_save_path = os.path.join(self.pixel_count_path, final_save_path)
                fig.savefig(final_save_path)
                plt.close()
            return len(x)
        return 0


    def get_outputs_nms(self, input_tensor, model, score_threshold = 0.5, iou_threshold = 0.5):
        mask_list = []
        label_list = []
        score_list =[]
        box_list = []
        with torch.no_grad():
            # forward pass of the image through the modle
            outputs = model(input_tensor)
        boxes = outputs[0]['boxes']
        labels = outputs[0]['labels']
        scores = outputs[0]['scores']
        masks = outputs[0]['masks']
        #print(boxes.shape,labels.shape, masks.shape)
        # Apply score threshold
        keep = scores > score_threshold
        boxes, labels, scores, masks = boxes[keep], labels[keep], scores[keep], masks[keep]
        #print(boxes.shape,labels.shape, masks.shape)
        #print(keep)
        keep = torchvision.ops.nms(boxes, scores, iou_threshold)
        boxes, labels, scores, masks = boxes[keep], labels[keep], scores[keep], masks[keep]
        #print(keep)
        #print(boxes.shape,labels.shape, masks.shape)
        scores = list(scores.detach().cpu().numpy())
        #print(scores)
        masks = list((masks>0.5).detach().cpu().numpy())
        #masks = list((masks>0.5).squeeze().detach().cpu().numpy())
        #print(masks.shape)
        boxes = [[(int(i[0]), int(i[1])), (int(i[2]), int(i[3]))]  for i in boxes.detach().cpu()]
        labels = list(labels.detach().cpu().numpy())
        labels = [self.class_names[i-1] for i in labels]
        
       # mask_list.append(masks)
        
        
        mask_list.append(masks)
        label_list.append(labels)
        score_list.append(scores)
        box_list.append(boxes)
        #print(len(masks),len(labels), len(scores), len(boxes))
        
        #print(masks[0].shape)
        #print(len(masks),len(labels), len(scores), len(boxes))
        return masks, boxes, labels, scores




    def get_outputs(self, input_tensor, model, threshold):
        mask_list = []
        label_list = []
        score_list =[]
        box_list = []
        with torch.no_grad():
            # forward pass of the image through the modle
            outputs = model(input_tensor)
            print(len(outputs))
        #for j in range(len(outputs)):
        j = 0
        scores = list(outputs[j]['scores'].detach().cpu().numpy())
        # print("\n scores", max(scores))
        # index of those scores which are above a certain threshold
        thresholded_preds_inidices = [scores.index(i) for i in scores if i > threshold]
        #print(thresholded_preds_inidices)
        scores = [scores[x] for x in thresholded_preds_inidices]
        # get the masks
        masks = (outputs[j]['masks']>0.5).squeeze().detach().cpu().numpy()
        # print("masks", masks)
        # discard masks for objects which are below threshold
        masks = [masks[x] for x in thresholded_preds_inidices]
        #print(masks[0].shape)
        # get the bounding boxes, in (x1, y1), (x2, y2) format
        boxes = [[(int(i[0]), int(i[1])), (int(i[2]), int(i[3]))]  for i in outputs[j]['boxes'].detach().cpu()]
        # discard bounding boxes below threshold value
        boxes = [boxes[x] for x in thresholded_preds_inidices]
        # get the classes labels
        labels = list(outputs[j]['labels'].detach().cpu().numpy())
        labels = [labels[x] for x in thresholded_preds_inidices]
        labels = [self.class_names[i-1] for i in labels]
        mask_list.append(masks)
        label_list.append(labels)
        score_list.append(scores)
        box_list.append(boxes)
        return masks, boxes, labels, scores
        #return mask_list, box_list, label_list, score_list
            


    def draw_segmentation_map(self, image, masks, boxes, labels):
        alpha = 1 
        beta = 0.6 # transparency for the segmentation map
        gamma = 0 # scalar added to each 
        segmentation_map = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        result_masks = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        masks = [ masks[i].squeeze() for i in range(len(masks))]
        for i in range(len(masks)):
            
            # TODO fix the color segmentation masks
            red_map = np.zeros_like(masks[i]).astype(np.uint8)
            green_map = np.zeros_like(masks[i]).astype(np.uint8)
            blue_map = np.zeros_like(masks[i]).astype(np.uint8)
            
            # apply a randon color mask to each object
            rect_color = (0,0,0)
            color = self.colors[random.randrange(0, len(self.colors))]
            print(self.class_to_colors[labels[i]])
            red_map[masks[i] == 1], green_map[masks[i] == 1], blue_map[masks[i] == 1]  = self.class_to_colors[labels[i]]
            result_masks[masks[i] == 1] = 255
            # combine all the masks into a single image
            # change the format of mask to W,H, C

            # segmentation_map = np.stack([red_map, green_map, blue_map], axis=2)
            #convert the original PIL image into NumPy format
            image = np.array(image)
            # convert from RGB to OpenCV BGR format
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # apply mask on the image
            # cv2.addWeighted(image, alpha, segmentation_map, beta, gamma, image)
            # draw the bounding boxes around the objects
            cv2.rectangle(image, boxes[i][0], boxes[i][1], color=rect_color, 
                        thickness=2)
            # Get the centre coords of the rectangle-plaque-detection/src/visualizat
            x1 = boxes[i][0][0]
            y1 = boxes[i][0][1]
            x2 = boxes[i][1][0]
            y2 = boxes[i][1][1]
            x = int((x1 + x2) / 2)
            y = int((y1+y2) / 2)

            
            # put the label text above the objects
            cv2.putText(image , labels[i], (x1, y1-20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color, 
                        thickness=2, lineType=cv2.LINE_AA)
            
            # Convert Back
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image, result_masks

    def prepare_input(self, image):
    
        image_float_np = np.float32(image) / 255

        # define the torchvision image transforms
        transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
        ])

        input_tensor = transform(image)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        input_tensor = input_tensor.to(device)
        # Add a batch dimension:
        input_tensor = input_tensor.unsqueeze(0)

        return input_tensor, image_float_np
    
    def predict(self, input_tensor, model, device, detection_threshold):
        outputs = model(input_tensor)
        # i- 1 zero indexing - the model outputs a non zero indexing format ( 1, 2, 3)
        pred_classes = [self.class_names[i-1] for i in outputs[0]['labels'].cpu().numpy()]
        pred_labels = outputs[0]['labels'].cpu().numpy()
        pred_scores = outputs[0]['scores'].detach().cpu().numpy()
        pred_bboxes = outputs[0]['boxes'].detach().cpu().numpy()
        
        boxes, classes, labels, indices = [], [], [], []
        for index in range(len(pred_scores)):
            if pred_scores[index] >= detection_threshold:
                boxes.append(pred_bboxes[index].astype(np.int32))
                classes.append(pred_classes[index])
                labels.append(pred_labels[index])
                indices.append(index)
        boxes = np.int32(boxes)
        return boxes, classes, labels, indices

    def draw_boxes(self, boxes, labels, classes, image):
        for i, box in enumerate(boxes):
            color = self.colors[labels[i]-1]
            cv2.rectangle(
                image,
                (int(box[0]), int(box[1])),
                (int(box[2]), int(box[3])),
                color, 2
            )
            cv2.putText(image, classes[i], (int(box[0]), int(box[1] + 30)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 1,
                        lineType=cv2.LINE_AA)
        return image

    def make_result_dirs(self, folder_name):
        #folder_name = self.wandb.name + "_" + folder_name
        
        save_path = os.path.join(self.result_save_dir, folder_name)
        results_path = os.path.join(save_path, "results")
        if not os.path.exists(results_path):
            os.makedirs(results_path)
        
        detections_path = os.path.join(save_path, "detections")
        if not os.path.exists(detections_path):
            os.makedirs(detections_path)
        
        masks_path = os.path.join(save_path, "masks")
        if not os.path.exists(masks_path):
            os.makedirs(masks_path)
        
        ablations_path = os.path.join(save_path, "ablations")
        if not os.path.exists(ablations_path):
            os.makedirs(ablations_path)
        
        pixel_count_path = os.path.join(save_path, "pixel_count")
        if not os.path.exists(pixel_count_path):
            os.makedirs(pixel_count_path)
        
        csv_name = folder_name + "_quantify.csv"
        quantify_path = os.path.join(save_path, csv_name)

        self.results_path = results_path
        self.masks_path = masks_path
        self.detections_path = detections_path
        self.ablations_path = ablations_path
        self.quantify_path = quantify_path
        self.pixel_count_path = pixel_count_path

    def quantify_plaques(self, df, wandb_result, img_name, result_img, result_masks, boxes, labels, scores, total_brown_pixels):
        '''This function will take masks image and generate attributes like plaque
        count, area, eccentricity'''
        csv_result = []
        for i in range(len(labels)):
            props = {}
            data = {}
            # Here x and y axis are flipped
            total_core_plaques = 0
            total_cg_plaques = 0
            total_diffused_plaques = 0
            total_caa_plaques = 0

            if len(boxes)!= 0:

                x1 = boxes[i][0][1]
                x2 =  boxes[i][1][1]
                y1 = boxes[i][0][0]
                y2 = boxes[i][1][0]

               
                cropped_img = result_img[x1:x2, y1:y2]
                cropped_img_mask = result_masks[x1:x2, y1:y2]
                unique_pixel = np.unique(cropped_img_mask)

                ret, bw_img = cv2.threshold(cropped_img_mask,0,255,cv2.THRESH_BINARY)

                kernel = np.ones((5,5),np.uint8)
                
                # Closing operation Dilation followed by erosion
                closing = cv2.morphologyEx(bw_img, cv2.MORPH_CLOSE, kernel)
                regions = regionprops(closing)
                if 0 in np.unique(closing):
                    mask_present=1
                else:
                    mask_present=0
                for props in regions:
                    if labels[i] == "Cored":
                        total_core_plaques+=1
                    elif labels[i] == "Coarse-Grained":
                        total_cg_plaques+=1
                    elif labels[i] == "Diffuse":
                        total_diffused_plaques+=1
                    elif labels[i] == "CAA":
                        total_caa_plaques+=1
                    
                    data_record = pd.DataFrame.from_records([{ 'image_name': img_name, 'label': labels[i] , 'confidence': scores[i],
                                                               'brown_pixels': total_brown_pixels,
                                                               'core': total_core_plaques, 'coarse_grained': total_cg_plaques, 'diffuse': total_diffused_plaques,
                                                               'caa':total_caa_plaques,
                                                               'centroid': props.centroid, 'eccentricity': props.eccentricity, 
                                                               'area': props.area, 'equivalent_diameter': props.equivalent_diameter, 'mask_present':mask_present}])
                    wandb_result.append([img_name, wandb.Image(cropped_img), wandb.Image(cropped_img_mask), labels[i], scores[i], 
                                         total_brown_pixels, props.centroid, props.eccentricity, props.area, props.equivalent_diameter,mask_present ])

                    df = pd.concat([df, data_record], ignore_index=True)
                   
        
        return df, wandb_result
    
 
    def generate_results(self):
        # This will help us create a different color for each class
        # Load Trained 
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.eval().to(device)

        tile_dirs = glob.glob(os.path.join(self.tile_dir, "*.npy"))
        
        slide_names = ['.'.join(file.split('.')[:-1]) for file in next(os.walk(slide_dir))[2]]
        out_dir ="/tmp"
        slide_names = slide_names[3:4]
        
        for slide_name in slide_names:
            num_tiles = tiling_nosaving.slide_tile_map(slide_name, slide_dir, tile_dir, tile_size, f=tiling_nosaving.crop_lambda(out_dir, slide_name))
            
            #print("\n", test_folder)
            #folder_name = os.path.basename(test_folder)
            #folder_name = test_folders.split("/")[-1].split(".")[0]
            #print(folder_name)

            #if folder_name == "labels":
            #    continue
            
            folder_name =  os.path.join(self.result_save_dir, slide_name)
            
            # make all necessary folders
            self.make_result_dirs(folder_name)
            
            #images = glob.glob(os.path.join(test_folder, "images", '*.png'))
            #actual_labels = glob.glob(os.path.join(test_folder, "labels", '*.png'))
            
            #test_dataset = build_features.AmyBDataset(test_folder, T.Compose([T.ToTensor()]))
            #collate_fn=lambda x: tuple(zip(*x))
            #exp_name = run.name
            #val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=collate_fn)
            i = 0
            df = pd.DataFrame()
            wandb_result = []
            total_core_plaques = 0
            total_neuritic_plaques = 0
            total_diffused_plaques = 0
            total_brown_pixels = 0
            total_image_pixels = 0
            #for i, (img, targets) in tqdm(enumerate(val_loader)):
            #for img, actual_mask_label in tqdm(zip(images,actual_labels)):
            #for zip_obj in tqdm(num_tiles):
                #image, img_name = zip(*zip_obj)
            for img_name, image  in tqdm(num_tiles):
                #img_name = os.path.basename(img).split('.')[0]
                #img_name 
                #image = img_np
                #print(image)
                #print(img_name)
                
                total_image_pixels+= image.shape[0] * image.shape[1]
                # Check if image has alpha channel
                if image.shape[2] == 4:
                    image = image[:,:, :3]

                input_tensor, image_float_np = self.prepare_input(image)
                masks, boxes, labels, scores = self.get_outputs_nms(input_tensor, self.model, self.detection_threshold, 0.5)
                #masks, boxes, labels, scores = self.get_outputs(input_tensor, self.model, self.detection_threshold)
                
                #f1_mean, labels_matched,actual_labels,pred_labels, scores =  evaluate_metrics(targets, masks, labels,scores )
                
                result_img, result_masks = self.draw_segmentation_map(image, masks, boxes, labels)

                total_brown_pixels+= self.get_brown_pixel_cnt(image, img_name)

                df, wandb_result = self.quantify_plaques(df, wandb_result, img_name, result_img, result_masks, boxes, labels, scores, total_brown_pixels)

                if self.save_result:
                    mask_img_name = img_name +  "_masks.png"
                    mask_save_path = os.path.join(self.masks_path, mask_img_name)

                    # Plot masks
                    cv2.imwrite(mask_save_path, result_masks)

                    # Plot detections
                    detection_img_name = img_name + "_detection.png"
                    detection_save_path = os.path.join(self.detections_path, detection_img_name)
                    bgr_img = cv2.cvtColor(result_img, cv2.COLOR_RGB2BGR)
                    cv2.imwrite(detection_save_path, bgr_img)

                    # Plot Results
                    plt.figure(figsize=(10,10))
                    plt.title("Model Prediction")

                    img_array = [result_img, result_masks]

                    for j in range(2):
                        plt.subplot(1, 2, j+1)
                        plt.imshow(img_array[j])
                    
                    result_img_name = img_name +  "_result.png"
                    result_save_path = os.path.join(self.results_path, result_img_name)
                    plt.savefig(result_save_path)
                    plt.close()

                # plt.show()
                if self.ablation_cam:
                    # Ablation CAM
                    boxes, classes, labels, indices = self.predict(input_tensor, model, device, self.detection_threshold)
                    target_layers = [model.backbone]
                    targets = [FasterRCNNBoxScoreTarget(labels=labels, bounding_boxes=boxes)]
                

                    cam = AblationCAM(model,
                                target_layers, 
                                use_cuda=torch.cuda.is_available(), 
                                reshape_transform=fasterrcnn_reshape_transform,
                                ablation_layer=AblationLayerFasterRCNN())

                    grayscale_cam = cam(input_tensor, targets=targets)
                    # Take the first image in the batch:
                    grayscale_cam = grayscale_cam[0, :]
                    cam_image = show_cam_on_image(image_float_np, grayscale_cam, use_rgb=True)
                    # And lets draw the boxes again:
                    image_with_bounding_boxes = self.draw_boxes(boxes, labels, classes, cam_image)

                    # plt.imshow(image_with_bounding_boxes)
                    plt.figure(figsize=(10,10))
                    plt.title("Ablation Cam")
                    ablation_list = [grayscale_cam, image_with_bounding_boxes]
                    for k in range(2):
                        plt.subplot(1, 2, k+1)
                        plt.imshow(ablation_list[k])


                    if self.save_result:
                        ablation_img_name = img_name +  "_ablation_cam.png"
                        save_path_ablation = os.path.join(self.ablations_path, ablation_img_name)
                        plt.savefig(save_path_ablation)
                i = i + 1
                # plt.show()

            #print("Total area of brown pixel", (total_brown_pixels))
            df.to_csv(self.quantify_path, index=False)
            #test_table = wandb.Table(data=wandb_result, columns=self.column_names)
            # self.wandb.log({'quantifications': test_table})


    


In [7]:
#slide_dir = '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/Full-Minerva-Data/AmyB-MFG'

tile_dir = '/gladstone/finkbeiner/steve/work/data/npsad_data/slide_masks'
slide_dir = '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/amy-def-mfg-test'
tile_size = (1024, 1024)

# watch out for '.DS_Store'
slide_names = ['.'.join(file.split('.')[:-1]) for file in next(os.walk(slide_dir))[2]]
#slide_names = ['.'.join(file.split('.')[:-1]) for file in next(os.walk(slide_dir))[2]]
slide_names= slide_names[3:4]
print(slide_names)
#tile_dir1 = '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi-backup/test-patients/images-npy'
#out_dir = '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/test/images'

#os.listdir(tile_dir)+os.listdir(tile_dir1)

#csv_test_amyb_mdf = pd.read_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Metadata/overlap_with_metadata_srna_flag/AmyB-MFG_hasClinicalDataflag_hassnRNAflag.csv")
#vips_img_dir_new = csv_test_amyb_mdf[csv_test_amyb_mdf["hasClinicalData"]==True]["path"].values

#test_folders = ["/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi-backup/test-patients/images-npy/XE05-025_1_AmyB_1.tiles.npy"]
#tiles = np.load(test_folders[0])

['XE11-008_1_AmyB_1']


In [9]:
#input_path = '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi-backup/test-patients/images-npy'
#model_name = "/home/mahirwar/Desktop/Monika/npsad_data/vivek/runpod_mrcnn_models/270ij5uq_epoch_66_step_800.ckpt"
model_name = "/home/mahirwar/Desktop/Monika/npsad_data/vivek/runpod_mrcnn_models/yp2mf3i8_epoch=108-step=872.ckpt"

test_config = dict(
    batch_size = 4,
    num_classes = 4
)

model_config = _default_mrcnn_config(num_classes=1 + test_config['num_classes']).config
backbone, rpn, roi_heads, transform1 = build_default(model_config, im_size=1024)

optim_config = dict(
    cls=torch.optim.Adam,
    defaults=dict(lr=0.00001,weight_decay=1e-6) 
)

#model  = LitMaskRCNN(optim_config,backbone,rpn,roi_heads,transform1)
model = LitMaskRCNN.load_from_checkpoint(model_name)
print(model)
#checkpoint = torch.load(model_name)
#model.load_state_dict(checkpoint["state_dict"])


#run = wandb.init(project="nps-ad-nature",  entity="monika-ahirwar")

explain = ExplainPredictions(model, model_input_path = model_name, slide_dir= slide_dir,tile_dir= tile_dir, 
                                detection_threshold=0.6, wandb=None, save_result=True, ablation_cam=False, save_thresholds=False)
explain.generate_results()

/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_32X8D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(ms

KeyboardInterrupt: 

In [2]:
%time
import torch
import numpy as np


    

import numpy as np

def split_tiles_into_batches(tiles, batch_size):
  tile_keys = np.array(list(tiles.keys()))
  np.random.shuffle(tile_keys)

  batches = np.array_split(tile_keys, len(tile_keys) // batch_size)
  return [{key: tiles[key] for key in batch} for batch in batches]

# Convert to a list of PyTorch tensors
tensor_list = [torch.from_numpy(array) for array in numpy_arrays]

# Create a batch tensor
batch_tensor = torch.stack(tensor_list, dim=0)

print(batch_tensor.shape)  # Output: torch.Size([5, 3, 4])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs
torch.Size([5, 1024, 1024, 3])


In [19]:
def prepare_input(image):
    # Normalize the image to [0, 1] range
    #image_float_np = image.astype(np.float32) / 255.0

    # Define the torchvision image transforms
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
    ])

    # Apply the transform to the image
    input_tensor = transform(image)

    # Move the tensor to the appropriate device (GPU if available)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_tensor = input_tensor.to(device)

    # Add a batch dimension
    input_tensor = input_tensor.unsqueeze(0)

    return input_tensor

In [20]:
%time
numpy_arrays = [np.random.randn(1024, 1024,3) for _ in range(10)]
y1 = [prepare_input(x) for x in numpy_arrays]


CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 15.7 µs


In [21]:
y1[0].shape

torch.Size([1, 3, 1024, 1024])

In [24]:
batch_tensor = torch.stack(y1, dim=0)

In [28]:
batch_tensor.squeeze().float().shape

torch.Size([10, 3, 1024, 1024])

In [3]:
# Sample data


tiles = dict()
for i, tile in enumerate(numpy_arrays):
    tiles["id"+str(i)]=tile

In [4]:
def split_tiles_into_batches(tiles, batch_size):
  tile_keys = np.array(list(tiles.keys()))
  #np.random.shuffle(tile_keys)

  batches = np.array_split(tile_keys, len(tile_keys) // batch_size)
  return [{key: tiles[key] for key in batch} for batch in batches]

In [6]:
%time
split_tiles_into_batches(tiles, 4)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10.5 µs


[{'id0': array([[[ 2.26876871, -0.7882767 , -0.35153369],
          [-0.67328631,  1.98605998,  0.32062553],
          [-0.76378849,  0.93223036,  0.22981568],
          ...,
          [ 0.64786049, -1.51805137,  0.19959122],
          [-0.6806723 ,  0.4516827 ,  0.0502512 ],
          [-0.51677451,  0.30672479,  0.99030258]],
  
         [[ 0.35753571,  0.36675981,  0.53211126],
          [-1.11919618, -1.13361862,  0.30315145],
          [ 0.32585512,  0.9006686 , -0.13809379],
          ...,
          [ 0.41931099, -0.22798427,  1.0055579 ],
          [ 1.97610254,  0.35448156, -0.42094608],
          [ 0.0657516 ,  1.04714575,  0.24941672]],
  
         [[-0.36803722, -0.38872912,  0.2206253 ],
          [ 0.51985274,  1.26584849,  0.88885603],
          [-1.51253314,  0.11523713,  1.63808601],
          ...,
          [ 0.29684815,  0.38980924,  0.78621031],
          [ 1.14581533, -0.15724259,  1.95432539],
          [-0.0756379 ,  1.25978853, -0.2218072 ]],
  
         ...,
  
 

In [8]:
model_name = "/home/mahirwar/Desktop/Monika/npsad_data/vivek/runpod_mrcnn_models/yp2mf3i8_epoch=108-step=872.ckpt"

test_config = dict(
    batch_size = 4,
    num_classes = 4
)

model_config = _default_mrcnn_config(num_classes=1 + test_config['num_classes']).config
backbone, rpn, roi_heads, transform1 = build_default(model_config, im_size=1024)

optim_config = dict(
    cls=torch.optim.Adam,
    defaults=dict(lr=0.00001,weight_decay=1e-6) 
)

#model  = LitMaskRCNN(optim_config,backbone,rpn,roi_heads,transform1)
model = LitMaskRCNN.load_from_checkpoint(model_name)
print(model)

/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_32X8D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(ms

LitMaskRCNN(
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(256,

/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'backbone' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['backbone'])`.
/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'rpn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['rpn'])`.
/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'roi_heads' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['roi_heads'])`.
/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/lightning/pytorch/u

In [30]:
from timeit import default_timer as timer 
start=timer()
with torch.no_grad():
    # forward pass of the image through the model
    outputs = model(batch_tensor.squeeze().float())
    
print(timer()-start)

0.779303299728781


In [11]:
batch_tensor.shape

torch.Size([5, 1024, 1024, 3])

In [ ]:
tensor_list = [torch.from_numpy(array) for array in numpy_arrays]

In [31]:
tile_dir =  '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-Data-Test/tiles-npy'
slide_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-Data/finkbeiner-124208/Images"
slide_name = "XE18-045_1_AmyB_1"
num_tiles = tiling_nosaving.slide_tile_map(slide_name, slide_dir, tile_dir, (1024,1024), f=tiling_nosaving.crop_lambda("", slide_name))

90364 210664


In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class NumpyArrayDataset(Dataset):
    def __init__(self, array_dict):
        """
        Args:
            arrays (list of np.ndarray): List of NumPy arrays.
            keys (list): List of keys corresponding to each array.
        """
        self.arrays = array_dict
        #self.keys = keys

    def __len__(self):
        return len(self.arrays)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the item to retrieve.
        
        Returns:
            dict: Contains the 'array' and 'key' for the item at index idx.
        """
        array = self.arrays[idx][1]
        key = self.arrays[idx][0]
        # Define the torchvision image transforms
        transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
        ])
        input_tensor = transform(array)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        input_tensor = input_tensor.to(device)
        
        # Add a batch dimension
        #input_tensor = input_tensor.unsqueeze(0)

        return {'array': input_tensor, 'key': key}

def create_dataloader(arrays, batch_size=1, shuffle=False, num_workers=0):
    dataset = NumpyArrayDataset(arrays)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    return dataloader


In [5]:
numpy_arrays = [(str(i),np.random.rand(1024, 1024, 3)) for i in range(10)]  # List of 10 arrays
num_tiles2= [x for x in numpy_arrays if (x[1].shape[1]==1024) & (x[1].shape[0]==1024) & (x[1].shape[2]==3)]

In [6]:
num_tiles2

[('0',
  array([[[0.77050041, 0.53691393, 0.46143573],
          [0.09947446, 0.52548565, 0.59948208],
          [0.52111452, 0.73016785, 0.23966148],
          ...,
          [0.71760929, 0.58009658, 0.85090446],
          [0.87195095, 0.67027553, 0.30936039],
          [0.55863025, 0.56703923, 0.09954643]],
  
         [[0.41628688, 0.26745726, 0.13166114],
          [0.19198478, 0.88153987, 0.74280607],
          [0.76152365, 0.30434157, 0.17598958],
          ...,
          [0.14409862, 0.33489766, 0.66132765],
          [0.58784789, 0.77957636, 0.30413996],
          [0.29052368, 0.05075177, 0.59304293]],
  
         [[0.59143523, 0.35743213, 0.79964766],
          [0.48614707, 0.34212942, 0.21557154],
          [0.96709334, 0.11585306, 0.75195802],
          ...,
          [0.78054874, 0.89407121, 0.88233303],
          [0.94520299, 0.29441047, 0.07561854],
          [0.59526724, 0.11926711, 0.80228467]],
  
         ...,
  
         [[0.83382365, 0.84775804, 0.84088932],
       

In [82]:
np.unique(num_tiles2)

array([ 252, 1024])

In [73]:
num_tiles[0][0]

'XE18-045_1_AmyB_1x_0_y_59392.png'

In [74]:
num_tiles[0][1]

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [242, 244, 241],
        [243, 243, 241],
        [246, 245, 243]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [239, 242, 233],
        [240, 241, 235],
        [241, 240, 236]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [235, 241, 231],
        [238, 240, 235],
        [238, 238, 236]]

In [75]:
len(num_tiles)

10635

In [86]:
num_tiles2

[]

In [94]:
numpy_arrays = [(str(i),np.random.rand(1024, 1024, 3)) for _ in range(10)]  # List of 10 arrays
keys = list(range(10))  # Corresponding keys

# Create DataLoader
dataloader = create_dataloader(num_tiles2, batch_size=6, shuffle=True)

for batch in dataloader:
    print("Batch of arrays shape:", batch['array'].shape)
    print("Batch of keys:", batch['key'])

Batch of arrays shape: torch.Size([6, 3, 1024, 1024])
Batch of keys: ['XE18-045_1_AmyB_1x_47104_y_132096.png', 'XE18-045_1_AmyB_1x_83968_y_68608.png', 'XE18-045_1_AmyB_1x_28672_y_76800.png', 'XE18-045_1_AmyB_1x_20480_y_79872.png', 'XE18-045_1_AmyB_1x_22528_y_89088.png', 'XE18-045_1_AmyB_1x_7168_y_165888.png']
Batch of arrays shape: torch.Size([6, 3, 1024, 1024])
Batch of keys: ['XE18-045_1_AmyB_1x_37888_y_56320.png', 'XE18-045_1_AmyB_1x_10240_y_84992.png', 'XE18-045_1_AmyB_1x_49152_y_65536.png', 'XE18-045_1_AmyB_1x_19456_y_137216.png', 'XE18-045_1_AmyB_1x_22528_y_109568.png', 'XE18-045_1_AmyB_1x_26624_y_161792.png']
Batch of arrays shape: torch.Size([6, 3, 1024, 1024])
Batch of keys: ['XE18-045_1_AmyB_1x_3072_y_89088.png', 'XE18-045_1_AmyB_1x_2048_y_68608.png', 'XE18-045_1_AmyB_1x_67584_y_153600.png', 'XE18-045_1_AmyB_1x_34816_y_74752.png', 'XE18-045_1_AmyB_1x_81920_y_63488.png', 'XE18-045_1_AmyB_1x_57344_y_83968.png']
Batch of arrays shape: torch.Size([6, 3, 1024, 1024])
Batch of keys

In [95]:
def get_outputs_nms(input_tensor, model, score_threshold = 0.5, iou_threshold = 0.5):
    start=timer()
    with torch.no_grad():
        # forward pass of the image through the model
        outputs = model(input_tensor)
    print(timer()-start)
    boxes = outputs[0]['boxes']
    labels = outputs[0]['labels']
    scores = outputs[0]['scores']
    masks = outputs[0]['masks']
    # Apply score threshold
    keep = scores > score_threshold
    boxes, labels, scores, masks = boxes[keep], labels[keep], scores[keep], masks[keep]
    keep = torchvision.ops.nms(boxes, scores, iou_threshold)
    boxes, labels, scores, masks = boxes[keep], labels[keep], scores[keep], masks[keep]
    scores = list(scores.detach().cpu().numpy())
    masks = list((masks>0.5).detach().cpu().numpy())
    boxes = [[(int(i[0]), int(i[1])), (int(i[2]), int(i[3]))]  for i in boxes.detach().cpu()]
    labels = list(labels.detach().cpu().numpy())
    labels = [self.class_names[i-1] for i in labels]
    return masks, boxes, labels, scores

In [96]:
get_outputs_nms(batch['array'], model, score_threshold = 0.5, iou_threshold = 0.5)

0.40081055788323283


([], [], [], [])

: 

In [9]:
import glob
import os
slide_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-Data/finkbeiner-124208/Images"
all_files = glob.glob(os.path.join(slide_dir,"*.mrxs"))

In [10]:
all_files

['/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-Data/finkbeiner-124208/Images/XE16-085_7_HE_1.mrxs',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-Data/finkbeiner-124208/Images/XE86-016_6_HE_1.mrxs',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-Data/finkbeiner-124208/Images/XE07-007_1_Syn_1.mrxs',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-Data/finkbeiner-124208/Images/XE19-029_1_Tau_1.mrxs',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-Data/finkbeiner-124208/Images/XE94-082_6_HE_1.mrxs',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-Data/finkbeiner-124208/Images/XE14-029_1_Sil_1.mrxs',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-Data/finkbeiner-124208/Images/XE17-038_1_Sil_3.mrxs',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-D

In [11]:
vips_img_fnames =  [i for i in all_files if i.endswith("_1_AmyB_1.mrxs")] # keeping only AMYB-MFG region images
vips_img_fnames =  [i.split("/")[-1].split(".")[0] for i in vips_img_fnames]

In [12]:
vips_img_fnames

['XE17-019_1_AmyB_1',
 'XE16-060_1_AmyB_1',
 'XE08-003_1_AmyB_1',
 'XE10-039_1_AmyB_1',
 'XE18-048_1_AmyB_1',
 'XE07-034_1_AmyB_1',
 'XE09-012_1_AmyB_1',
 'XE09-041_1_AmyB_1',
 'XE12-015_1_AmyB_1',
 'XE19-023_1_AmyB_1',
 'XE13-020_1_AmyB_1',
 'XE17-052_1_AmyB_1',
 'XE18-005_1_AmyB_1',
 'XE09-018_1_AmyB_1',
 'XE10-010_1_AmyB_1',
 'XE18-063_1_AmyB_1',
 'XE11-008_1_AmyB_1',
 'XE16-053_1_AmyB_1',
 'XE10-009_1_AmyB_1',
 'XE08-026_1_AmyB_1',
 'XE10-030_1_AmyB_1',
 'XE18-045_1_AmyB_1',
 'XE15-039_1_AmyB_1',
 'XE17-050_1_AmyB_1',
 'XE16-055_1_AmyB_1',
 'XE19-042_1_AmyB_1',
 'XE09-034_1_AmyB_1',
 'XE10-045_1_AmyB_1',
 'XE10-002_1_AmyB_1',
 'XE18-079_1_AmyB_1',
 'XE13-005_1_AmyB_1',
 'XE10-014_1_AmyB_1',
 'XE05-025_1_AmyB_1',
 'XE17-003_1_AmyB_1',
 'XE07-033_1_AmyB_1',
 'XE15-054_1_AmyB_1',
 'XE16-002_1_AmyB_1',
 'XE11-003_1_AmyB_1',
 'XE12-032_1_AmyB_1',
 'XE09-028_1_AmyB_1',
 'XE15-051_1_AmyB_1',
 'XE07-076_1_AmyB_1',
 'XE07-047_1_AmyB_1',
 'XE10-008_1_AmyB_1',
 'XE09-049_1_AmyB_1',
 'XE16-081

In [14]:
output = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/reports/New-Minerva-Data-output/yp2mf3i8_epoch=108-step=872.ckpt"
output1 = glob.glob(os.path.join(output,"*.csv"))

In [15]:
to_exclude =  [i.split("/")[-1].replace("_quantify.csv","") for i in output1]

In [17]:
to_exclude

['XE17-052_1_AmyB_1',
 'XE09-041_1_AmyB_1',
 'XE10-010_1_AmyB_1',
 'XE18-005_1_AmyB_1',
 'XE10-009_1_AmyB_1',
 'XE10-039_1_AmyB_1',
 'XE16-060_1_AmyB_1',
 'XE18-048_1_AmyB_1',
 'XE15-039_1_AmyB_1',
 'XE12-015_1_AmyB_1',
 'XE17-050_1_AmyB_1',
 'XE11-008_1_AmyB_1',
 'XE18-063_1_AmyB_1',
 'XE10-030_1_AmyB_1',
 'XE19-023_1_AmyB_1',
 'XE13-020_1_AmyB_1',
 'XE08-026_1_AmyB_1',
 'XE17-019_1_AmyB_1',
 'XE18-045_1_AmyB_1',
 'XE08-003_1_AmyB_1',
 'XE07-034_1_AmyB_1',
 'XE09-012_1_AmyB_1',
 'XE09-018_1_AmyB_1']

In [18]:
keep =[ x for x in vips_img_fnames if x not in to_exclude]

In [19]:
len(keep)

301

In [ ]:
keep

In [29]:
import pandas as pd
df = pd.DataFrame({"slide_name":keep[150:]})

In [30]:
df

,slide_name
0,XE12-009_1_AmyB_1
1,XE17-049_1_AmyB_1
2,XE16-069_1_AmyB_1
3,XE07-072_1_AmyB_1
4,XE17-038_1_AmyB_1
...,...
146,XE19-032_1_AmyB_1
147,XE09-004_1_AmyB_1
148,XE05-123_1_AmyB_1
149,XE18-038_1_AmyB_1


In [28]:
df.to_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-Data-Test/files_to_run_vivek.csv")

In [31]:
df.to_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-Data-Test/files_to_run_monika.csv")

In [3]:
f = open("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/New-Minerva-Data-Test/filenotrun.txt", "r")
files = f.read()

In [7]:
files = files.split(" ")

In [8]:
files

['XE16-053_1_AmyB_1',
 'XE08-039_1_AmyB_1',
 'XE08-039_1_AmyB_1',
 'XE17-015_1_AmyB_1',
 'XE16-019_1_AmyB_1',
 'XE15-005_1_AmyB_1',
 'XE19-009_1_AmyB_1',
 'XE17-004_1_AmyB_1',
 'XE19-049_1_AmyB_1',
 'XE09-063_1_AmyB_1',
 'XE08-005_1_AmyB_1',
 'XE10-034_1_AmyB_1',
 'XE09-007_1_AmyB_1',
 'XE12-042_1_AmyB_1',
 'XE06-101_1_AmyB_1',
 'XE15-033_1_AmyB_1',
 'XE08-042_1_AmyB_1',
 'XE05-123_1_AmyB_1',
 'XE16-053_1_AmyB_1',
 'XE10-045_1_AmyB_1',
 'XE10-002_1_AmyB_1',
 'XE18-079_1_AmyB_1',
 'XE13-005_1_AmyB_1',
 'XE10-014_1_AmyB_1',
 'XE05-025_1_AmyB_1',
 'XE17-003_1_AmyB_1',
 'XE07-033_1_AmyB_1',
 'XE15-054_1_AmyB_1',
 'XE16-002_1_AmyB_1',
 'XE11-003_1_AmyB_1',
 'XE12-032_1_AmyB_1',
 'XE09-028_1_AmyB_1',
 'XE15-051_1_AmyB_1',
 'XE07-076_1_AmyB_1',
 'XE07-047_1_AmyB_1',
 'XE12-032_1_AmyB_1',
 'XE09-028_1_AmyB_1',
 'XE15-051_1_AmyB_1',
 'XE07-076_1_AmyB_1',
 'XE07-047_1_AmyB_1',
 'XE10-008_1_AmyB_1',
 'XE09-049_1_AmyB_1']

In [11]:
csv_files = glob.glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/reports/New-Minerva-Data-output/yp2mf3i8_epoch=108-step=872.ckpt","*.csv"))

In [13]:
csv_files = [x.split("/")[-1].replace("_quantify.csv","") for x in csv_files]

In [17]:
files2run = [x for x in files if x not in csv_files]

In [18]:
pd.DataFrame({"WSI_name":files2run}).to_csv(c)